In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import re
import time
import requests


def extraer_sku_y_gramos(url):
    """Usa Selenium para abrir la página y obtener SKU y gramos/ml (Shopify carga con JS)."""
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    try:
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        # Esperar a que cargue al menos el bloque principal del producto
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.product__sku, p.custom_product__text"))
        )

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # --- SKU ---
        sku = "N/A"
        p_sku = soup.find("p", class_="product__sku")
        if p_sku:
            texto = p_sku.get_text(strip=True)
            match = re.search(r"SKU[:\s]*([A-Za-z0-9\-]+)", texto)
            if match:
                sku = match.group(1)
            else:
                sku = texto

        # --- Gramos / ml ---
        gramos = "N/A"
        p_gramos = soup.find("p", class_="custom_product__text product__text")
        if p_gramos:
            texto_g = p_gramos.get_text(strip=True)
            match_g = re.search(r"(\d+(?:[.,]\d+)?\s*(?:g|ml|gr|GR|ML))", texto_g, re.IGNORECASE)
            if match_g:
                gramos = match_g.group(1).replace(",", ".").lower()

        driver.quit()
        return sku, gramos

    except Exception as e:
        print(f"⚠️ Error en {url}: {e}")
        try:
            driver.quit()
        except:
            pass
        return "N/A", "N/A"


def rostro():
    """Extrae TODOS los productos de la categoría 'Cuidado de la Piel' de Nala y los devuelve como un DataFrame."""
    
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.get("https://nala.es/collections/cuidado-de-la-piel")

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.grid__item"))
    )

    # Scroll dinámico
    last_count = 0
    stable_rounds = 0
    while stable_rounds < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "li.grid__item")
        if len(products) == last_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
        last_count = len(products)

    print(f"Total productos detectados en la página: {last_count}")

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    items = soup.select("li.grid__item")
    print(f"Productos encontrados en HTML: {len(items)}")

    productos = []
    categorias = defaultdict(int)

    for item in items:
        enlace = item.find("a", href=re.compile("/products/"))
        if not enlace:
            continue
        url_producto = "https://nala.es" + enlace.get("href")

        # ====== 🔍 NUEVA EXTRACCIÓN ROBUSTA DEL NOMBRE ======
        nombre = None

        nombre_tag = item.find(attrs={"class": re.compile("(title|heading)", re.IGNORECASE)})
        if nombre_tag and nombre_tag.get_text(strip=True):
            nombre = nombre_tag.get_text(strip=True)

        if not nombre:
            hidden_name = item.find("span", class_=re.compile("visually-hidden", re.IGNORECASE))
            if hidden_name:
                nombre = hidden_name.get_text(strip=True)

        if not nombre:
            nombre = enlace.get_text(strip=True)

        if not nombre:
            nombre = "N/A"
        # =====================================================

        # Precio
        precio_tag = item.find("div", class_=re.compile("price", re.IGNORECASE))
        precio = "N/A"
        precio_num= None 
        if precio_tag:
            precio_texto = precio_tag.get_text(strip=True)
            precio_match = re.search(r"\d+,\d{2}", precio_texto)
            if precio_match:
                precio_num = float(precio_match.group().replace(",", "."))
                precio = f"{precio_match.group()} €"

        # Ingrediente
        ingrediente = re.search(
            r"(granada|café|vainilla|aloe|argán|karité|cereza|grapefruit|Cóctel|Sandía|fresa|jojoba|caléndula|manzanilla|aguacate|cáñamo|rosas|lavanda|menta|coco|limón|melocotón|piña|mango|pepino|pomelo|uvas|naranja)",
            nombre,
            re.IGNORECASE
        )

        nombre_lower = nombre.lower()
        if "exfoliante" in nombre_lower and "labial" in nombre_lower:
            categoria = "Exfoliante Labial"
        elif "exfoliante" in nombre_lower and "facial" in nombre_lower:
            categoria = "Exfoliante Facial"
        elif "ojos" in nombre_lower:
            categoria = "Contorno de Ojos"
        elif "crema" in nombre_lower and "facial" in nombre_lower:
            categoria = "Crema Facial"
        elif "crema" in nombre_lower and "piel" in nombre_lower:
            categoria = "Crema Facial"
        elif "suero" in nombre_lower:
            categoria = "Serum Facial"
        elif "sérum" in nombre_lower and "facial" in nombre_lower:
            categoria = "Serum Facial"
        elif "set" in nombre_lower:
            categoria = "Set facial"
        elif "spf" in nombre_lower or "protector solar" in nombre_lower:
            categoria = "Protector Solar Facial"
        elif "bálsamo" in nombre_lower:
            categoria = "Bálsamo"
        elif "espuma" in nombre_lower:
            categoria = "Limpiador facial"
        elif "gel" in nombre_lower and "limpiador" in nombre_lower:
            categoria = "Limpiador facial"
        elif "tónico" in nombre_lower or "tonico" in nombre_lower:
            categoria = "Tónico Facial"
        elif "mascarilla" in nombre_lower:
            categoria = "Mascarilla Facial"
        elif "micelar" in nombre_lower:
            categoria = "Limpiador Facial"
        elif "leche" in nombre_lower:
            categoria = "Leche Limpiador Facial"
        elif "serum" in nombre_lower and "facial" in nombre_lower:
            categoria = "Serum Facial"
        elif "mascarilla" in nombre_lower:
            categoria = "Mascarilla Facial"
        elif "crema" in nombre_lower and "rostro" in nombre_lower:
            categoria = "Crema Facial"
        else:
            categoria = "Otro"

        categorias[categoria] += 1

        productos.append({
            "SKU": "N/A",   # se completará después
            "nombre": nombre,
            "categoria_general": "Rostro",
            "categoria": categoria,
            "precio": precio,
            "ingrediente_clave": ingrediente.group(1).capitalize() if ingrediente else "N/A",
            "país": "España",  # 🇪🇸 nuevo campo
            "url": url_producto,
        })

    # === Extraer SKUs y gramos en paralelo ===
    print("Extrayendo SKUs y gramos/ml de cada producto...")
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extraer_sku_y_gramos, p["url"]): i for i, p in enumerate(productos)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                sku, gramos = future.result()
                productos[idx]["SKU"] = sku
                productos[idx]["gramos/ml"] = gramos
            except Exception:
                productos[idx]["SKU"] = "N/A"
                productos[idx]["gramos/ml"] = "N/A"

    df = pd.DataFrame(productos)
    print(f"Total productos finales: {len(df)}")
    # === Convertir precio y gramos/ml a columnas numéricas ===
    df["precio"] = (
        df["precio"]
        .str.replace("€", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
        )

    df["gramos/ml"] = (
        df["gramos/ml"]
        .str.extract(r"(\d+(?:\.\d+)?)")[0]   # Extrae el número
        .astype(float)                        # Convierte a decimal
        .round()                              # Redondea si hay decimales
        .astype("Int64")                      # Convierte a entero permitiendo NaN
    )

    return df



In [34]:
df_rostro = rostro()


Total productos detectados en la página: 96
Productos encontrados en HTML: 96
Extrayendo SKUs y gramos/ml de cada producto...
Total productos finales: 92


In [35]:
df_rostro

,SKU,nombre,categoria_general,categoria,precio,ingrediente_clave,país,url,gramos/ml
0,70402,Exfoliante labial - Granada,Rostro,Exfoliante Labial,3.90,Granada,España,https://nala.es/products/exfoliante-labial-gra...,25
1,64223,Bálsamo labial - Vainilla,Rostro,Bálsamo,3.90,Vainilla,España,https://nala.es/products/balsamo-labial-vainilla,11
2,64836,Bálsamo labial - Cereza,Rostro,Bálsamo,3.90,Cereza,España,https://nala.es/products/balsamo-labial-cereza,11
3,64222,Bálsamo labial - Grapefruit,Rostro,Bálsamo,3.90,Grapefruit,España,https://nala.es/products/balsamo-labial-pomelo,11
4,64221,Bálsamo labial - Cóctel de frutas,Rostro,Bálsamo,3.90,Cóctel,España,https://nala.es/products/balsamo-labial-coctel...,11
...,...,...,...,...,...,...,...,...,...
87,1300,Set para el cuidado de la piel madura - Antien...,Rostro,Set facial,35.95,N/A,España,https://nala.es/products/set-para-el-cuidado-d...,<NA>
88,64830,Serum Contorno de Ojos Luminoso - Ojeras y Bol...,Rostro,Contorno de Ojos,12.90,N/A,España,https://nala.es/products/aclarar-ojeras-ojeras,30
89,2016,Set de tratamiento para pieles propensas al acné,Rostro,Set facial,14.95,N/A,España,https://nala.es/products/set-de-tratamiento-pa...,<NA>
90,30675,Crema facial antiacné - Pieles propensas al ac...,Rostro,Crema Facial,15.90,Aloe,España,https://nala.es/products/crema-facial-antiacne...,50


In [ ]:
def extraer_sku_y_gramos(url):
    """Usa Selenium para abrir la página y obtener SKU y gramos/ml (Shopify carga con JS)."""
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    try:
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        # Esperar a que cargue el SKU o la descripción del producto
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.product__sku, p.custom_product__text"))
        )

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # --- SKU ---
        sku = "N/A"
        p_sku = soup.find("p", class_="product__sku")
        if p_sku:
            texto = p_sku.get_text(strip=True)
            match = re.search(r"SKU[:\s]*([A-Za-z0-9\-]+)", texto)
            if match:
                sku = match.group(1)
            else:
                sku = texto

        # --- Gramos / ml ---
        gramos = "N/A"
        p_gramos = soup.find("p", class_="custom_product__text product__text")
        if p_gramos:
            texto_g = p_gramos.get_text(strip=True)
            match_g = re.search(r"(\d+(?:[.,]\d+)?\s*(?:g|ml|gr|GR|ML))", texto_g, re.IGNORECASE)
            if match_g:
                gramos = match_g.group(1).replace(",", ".").lower()

        driver.quit()
        return sku, gramos

    except Exception as e:
        print(f"⚠️ Error en {url}: {e}")
        try:
            driver.quit()
        except:
            pass
        return "N/A", "N/A"


def cabello():
    """Extrae TODOS los productos de la categoría 'Cabello' de Nala y devuelve un DataFrame."""
    
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.get("https://nala.es/collections/pelo")

    # Esperar a que se carguen los primeros productos
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.grid__item"))
    )

    # --- Scroll dinámico hasta que no carguen más ---
    last_count = 0
    stable_rounds = 0
    while stable_rounds < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "li.grid__item")
        if len(products) == last_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
        last_count = len(products)

    print(f"Total productos detectados en la página: {last_count}")

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    items = soup.select("li.grid__item")
    print(f"Productos encontrados en HTML: {len(items)}")

    productos = []
    categorias = defaultdict(int)

    for item in items:
        enlace = item.find("a", href=re.compile("/products/"))
        if not enlace:
            continue
        url_producto = "https://nala.es" + enlace.get("href")

        # ====== 🔍 Extracción robusta del nombre ======
        nombre = None
        nombre_tag = item.find(attrs={"class": re.compile("(title|heading)", re.IGNORECASE)})
        if nombre_tag and nombre_tag.get_text(strip=True):
            nombre = nombre_tag.get_text(strip=True)

        if not nombre:
            hidden_name = item.find("span", class_=re.compile("visually-hidden", re.IGNORECASE))
            if hidden_name:
                nombre = hidden_name.get_text(strip=True)

        if not nombre:
            nombre = enlace.get_text(strip=True)

        if not nombre:
            nombre = "N/A"
        # =====================================================

        # Precio
        precio_tag = item.find("div", class_=re.compile("price", re.IGNORECASE))
        precio = "N/A"
        if precio_tag:
            precio_texto = precio_tag.get_text(strip=True)
            precio_match = re.search(r"\d+,\d{2}", precio_texto)
            if precio_match:
                precio = precio_match.group() + " €"

        # Ingrediente
        ingrediente = re.search(
            r"(granada|lemongrass|miel|cafeina|camomila|violeta absoluto|vainilla|aloe|argán|karité|jojoba|caléndula|manzanilla|aguacate|cáñamo|rosas|lavanda|menta|coco|limón|melocotón|piña|mango|pepino|pomelo|uvas|naranja)",
            nombre,
            re.IGNORECASE
        )

        # Clasificación por categoría
        nombre_lower = nombre.lower()
        if "champú" in nombre_lower and "sólido" in nombre_lower:
            categoria = "Champu Solido"
        elif "champu" in nombre_lower or "champú" in nombre_lower:
            categoria = "Champú"
        elif "tratamiento" in nombre_lower:
            categoria = "Tratamiento Capilar"
        elif "mascarilla" in nombre_lower:
            categoria = "Mascarilla"
        elif "acondicionador" in nombre_lower:
            categoria = "Acondicionador"
        elif "tónico" in nombre_lower or "tonico" in nombre_lower:
            categoria = "Tónico Capilar"
        elif "aceite" in nombre_lower:
            categoria = "Aceite Capilar"
        elif "bálsamo" in nombre_lower:
            categoria = "Balsamo capilar"
        elif "set" in nombre_lower:
            categoria = "Set Capilar"
        elif "cera" in nombre_lower:
            categoria = "Cera Capilar"
        elif "spray" in nombre_lower:
            categoria = "Spray Capilar"
        else:
            categoria = "Otro"

        categorias[categoria] += 1

        productos.append({
            "SKU": "N/A",        # se completará después
            "gramos/ml": "N/A",  # nuevo campo
            "nombre": nombre,
            "categoria_general": "Cabello",
            "categoria": categoria,
            "precio": precio,
            "ingrediente_clave": ingrediente.group(1).capitalize() if ingrediente else "N/A",
            "país": "España",    # nuevo campo
            "url": url_producto,
        })

    # === Extraer SKUs y gramos en paralelo ===
    print("Extrayendo SKUs y gramos/ml de cada producto...")
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extraer_sku_y_gramos, p["url"]): i for i, p in enumerate(productos)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                sku, gramos = future.result()
                productos[idx]["SKU"] = sku
                productos[idx]["gramos/ml"] = gramos
            except Exception:
                productos[idx]["SKU"] = "N/A"
                productos[idx]["gramos/ml"] = "N/A"

    df = pd.DataFrame(productos)
    print(f"Total productos finales: {len(df)}")
    # === Convertir precio y gramos/ml a columnas numéricas ===
    df["precio"] = (
        df["precio"]
        .str.replace("€", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
        )

    df["gramos/ml"] = (
        df["gramos/ml"]
        .str.extract(r"(\d+(?:\.\d+)?)")[0]   # Extrae el número
        .astype(float)                        # Convierte a decimal
        .round()                              # Redondea si hay decimales
        .astype("Int64")                      # Convierte a entero permitiendo NaN
    )

    return df



In [37]:
df_cabello = cabello()
df_cabello


Total productos detectados en la página: 49
Productos encontrados en HTML: 49
Extrayendo SKUs y gramos/ml de cada producto...
Total productos finales: 45


,SKU,gramos/ml,nombre,categoria_general,categoria,precio,ingrediente_clave,país,url
0,30692,200,Champú Cabello Graso - Camomila,Cabello,Champú,7.9,Camomila,España,https://nala.es/products/aceite-cabello-champu...
1,30702,200,Mascarilla capilar - Coco,Cabello,Mascarilla,8.9,Coco,España,https://nala.es/products/mascarilla-capilar-coco
2,30798,200,Mascarilla Capilar Voluminizante y Brillante -...,Cabello,Mascarilla,8.9,Argán,España,https://nala.es/products/mascarilla-capilar-vo...
3,64285,75,Tónico Capilar Fortificante y Estimulante del ...,Cabello,Tónico Capilar,8.9,N/A,España,https://nala.es/products/fortificante-estimula...
4,64307,200,Champú Púrpura Cabello Rubio - Extracto Absolu...,Cabello,Champú,7.9,N/A,España,https://nala.es/products/champu-rubio-purpura-...
5,64287,200,Acondicionador Púrpura Cabello Rubio - Aceite ...,Cabello,Acondicionador,8.9,Coco,España,https://nala.es/products/acondicionador-purpur...
6,70438,200,Acondicionador Hidratante - Jojoba y Rosas,Cabello,Acondicionador,8.9,Jojoba,España,https://nala.es/products/acondicionador-hidrat...
7,64288,200,Mascarilla Morada Cabello Rubio - Aceite de Co...,Cabello,Mascarilla,6.9,Coco,España,https://nala.es/products/purple-mask-blonde-ha...
8,70454,58,Champú sólido para cabello graso - Lemongrass,Cabello,Champu Solido,7.9,Lemongrass,España,https://nala.es/products/solid-shampoo-oily-ha...
9,64308,200,"Champú Hidratante Intensivo - 50% Miel, Rosas,...",Cabello,Champú,9.9,Miel,España,https://nala.es/products/champu-hidratante-int...


In [38]:
def extraer_sku_y_gramos(url):
    """Usa Selenium para abrir la página y obtener SKU y gramos/ml (Shopify carga con JS)."""
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    try:
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        # Esperar a que cargue el contenido
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.product__sku, p.custom_product__text"))
        )

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # --- SKU ---
        sku = "N/A"
        p_sku = soup.find("p", class_="product__sku")
        if p_sku:
            texto = p_sku.get_text(strip=True)
            match = re.search(r"SKU[:\s]*([A-Za-z0-9\-]+)", texto)
            if match:
                sku = match.group(1)
            else:
                sku = texto

        # --- Gramos / ml ---
        gramos = "N/A"
        p_gramos = soup.find("p", class_="custom_product__text product__text")
        if p_gramos:
            texto_g = p_gramos.get_text(strip=True)
            match_g = re.search(r"(\d+(?:[.,]\d+)?\s*(?:g|ml|gr|GR|ML))", texto_g, re.IGNORECASE)
            if match_g:
                gramos = match_g.group(1).replace(",", ".").lower()

        driver.quit()
        return sku, gramos

    except Exception as e:
        print(f"⚠️ Error en {url}: {e}")
        try:
            driver.quit()
        except:
            pass
        return "N/A", "N/A"


def corporal():
    """Extrae TODOS los productos de la categoría 'Corporal' de Nala y devuelve un DataFrame."""

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.get("https://nala.es/collections/cuidado-corporal")

    # Esperar que carguen los primeros productos
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.grid__item"))
    )

    # --- Scroll dinámico ---
    last_count = 0
    stable_rounds = 0
    while stable_rounds < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "li.grid__item")
        if len(products) == last_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
        last_count = len(products)

    print(f"Total productos detectados en la página: {last_count}")

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    items = soup.select("li.grid__item")
    print(f"Productos encontrados en HTML: {len(items)}")

    productos = []
    categorias = defaultdict(int)

    for item in items:
        enlace = item.find("a", href=re.compile("/products/"))
        if not enlace:
            continue
        url_producto = "https://nala.es" + enlace.get("href")

        # ====== 🔍 Extracción robusta del nombre ======
        nombre = None
        nombre_tag = item.find(attrs={"class": re.compile("(title|heading|card__title)", re.IGNORECASE)})
        if nombre_tag and nombre_tag.get_text(strip=True):
            nombre = nombre_tag.get_text(strip=True)

        if not nombre:
            hidden_name = item.find("span", class_=re.compile("visually-hidden", re.IGNORECASE))
            if hidden_name:
                nombre = hidden_name.get_text(strip=True)

        if not nombre:
            nombre = enlace.get_text(strip=True)

        if not nombre:
            nombre = "N/A"
        # =====================================================

        # Precio
        precio_tag = item.find("div", class_=re.compile("price", re.IGNORECASE))
        precio = "N/A"
        if precio_tag:
            precio_texto = precio_tag.get_text(strip=True)
            precio_match = re.search(r"\d+,\d{2}", precio_texto)
            if precio_match:
                precio = precio_match.group() + " €"

        # Ingrediente
        ingrediente = re.search(
            r"(granada|café|vainilla|aloe|argán|karité|jojoba|caléndula|manzanilla|aguacate|cáñamo|rosas|lavanda|menta|coco|limón|melocotón|piña|mango|pepino|pomelo|uvas|naranja)",
            nombre,
            re.IGNORECASE
        )

        # Clasificación por categoría
        nombre_lower = nombre.lower()
        if "manos" in nombre_lower:
            categoria = "Crema de Manos"
        elif "spray" in nombre_lower:
            categoria = "Spray corporal"
        elif "pack" in nombre_lower:
            categoria = "Pack Desodorante"
        elif "desodorante" in nombre_lower:
            categoria = "Desodorante"
        elif "gel" in nombre_lower:
            categoria = "Gel Corporal"
        elif "bomba" in nombre_lower:
            categoria = "Bomba de Baño"
        elif "crema" in nombre_lower:
            categoria = "Crema Corporal"
        elif "aceite" in nombre_lower:
            categoria = "Aceite Corporal"
        elif "sorbete" in nombre_lower:
            categoria = "Sorbete Corporal"
        elif "exfoliante" in nombre_lower:
            categoria = "Exfoliante Corporal"
        elif "bruma" in nombre_lower:
            categoria = "Bruma Corporal"
        elif "spf30" in nombre_lower:
            categoria = "Protector Solar Corporal"
        elif "spf50" in nombre_lower:
            categoria = "Protector Solar Corporal"
        elif "jabón" in nombre_lower:
            categoria = "Jabón Natural"
        elif "manteca" in nombre_lower:
            categoria = "Manteca Corporal"
        elif "bálsamo" in nombre_lower:
            categoria = "Balsamo Labial"
        elif "íntimo" in nombre_lower:
            categoria = "Gel Intimo"
        elif "leche" in nombre_lower:
            categoria = "Leche Corporal"
        else:
            categoria = "Otro"

        categorias[categoria] += 1

        productos.append({
            "SKU": "N/A",        # se completará después
            "gramos/ml": "N/A",  # nuevo campo
            "nombre": nombre,
            "categoria_general": "Corporal",
            "categoria": categoria,
            "precio": precio,
            "ingrediente_clave": ingrediente.group(1).capitalize() if ingrediente else "N/A",
            "país": "España",    # nuevo campo
            "url": url_producto,
        })

    # === Extraer SKUs y gramos en paralelo ===
    print("Extrayendo SKUs y gramos/ml de cada producto...")
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extraer_sku_y_gramos, p["url"]): i for i, p in enumerate(productos)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                sku, gramos = future.result()
                productos[idx]["SKU"] = sku
                productos[idx]["gramos/ml"] = gramos
            except Exception:
                productos[idx]["SKU"] = "N/A"
                productos[idx]["gramos/ml"] = "N/A"

    df = pd.DataFrame(productos)
    print(f"Total productos finales: {len(df)}")
    # === Convertir precio y gramos/ml a columnas numéricas ===
    df["precio"] = (
        df["precio"]
        .str.replace("€", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
        )

    df["gramos/ml"] = (
        df["gramos/ml"]
        .str.extract(r"(\d+(?:\.\d+)?)")[0]   # Extrae el número
        .astype(float)                        # Convierte a decimal
        .round()                              # Redondea si hay decimales
        .astype("Int64")                      # Convierte a entero permitiendo NaN
    )

    return df


In [39]:
df_coproral= corporal()
df_coproral

Total productos detectados en la página: 172
Productos encontrados en HTML: 172
Extrayendo SKUs y gramos/ml de cada producto...
Total productos finales: 168


,SKU,gramos/ml,nombre,categoria_general,categoria,precio,ingrediente_clave,país,url
0,35421,200,Exfoliante de Sal – Frutos Rojos & Bergamota,Corporal,Exfoliante Corporal,9.9,N/A,España,https://nala.es/products/exfoliante-sal-frutos...
1,35438,200,Exfoliante de Sal – Pimienta Rosa & Ámbar,Corporal,Exfoliante Corporal,9.9,N/A,España,https://nala.es/products/exfoliante-sal-pimien...
2,35452,200,Exfoliante de Sal – Pera & Sandía,Corporal,Exfoliante Corporal,9.9,N/A,España,https://nala.es/products/exfoliante-sal-pera-s...
3,35445,200,Exfoliante de Sal – Arándanos & Uvas,Corporal,Exfoliante Corporal,9.9,Uvas,España,https://nala.es/products/exfoliante-sal-aranda...
4,35537,125,Manteca Corporal - Pera & Sandía,Corporal,Manteca Corporal,9.9,N/A,España,https://nala.es/products/manteca-corporal-pera...
...,...,...,...,...,...,...,...,...,...
163,64825,100,Mascarilla Rehab Manos y Pies - Cera de zumaqu...,Corporal,Crema de Manos,9.9,Jojoba,España,https://nala.es/products/rehabilitacion-manos-...
164,64824,60,"Polvo Desodorante para Pies - Arcilla, Eucalip...",Corporal,Desodorante,8.9,N/A,España,https://nala.es/products/polvo-desodorante-pie...
165,65102,150,Sorbete corporal - Cóctel de frutas,Corporal,Sorbete Corporal,8.9,N/A,España,https://nala.es/products/body-sorbet-fruits-co...
166,64214,200,Leche corporal - Coco y vainilla,Corporal,Leche Corporal,8.9,Coco,España,https://nala.es/products/body-milk-coco-vainilla


In [40]:
def extraer_sku_y_gramos(url):
    """Usa Selenium para abrir la página y obtener SKU y gramos/ml (Shopify carga con JS)."""
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    try:
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        # Esperar que carguen los elementos relevantes
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.product__sku, p.custom_product__text"))
        )

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # --- SKU ---
        sku = "N/A"
        p_sku = soup.find("p", class_="product__sku")
        if p_sku:
            texto = p_sku.get_text(strip=True)
            match = re.search(r"SKU[:\s]*([A-Za-z0-9\-]+)", texto)
            if match:
                sku = match.group(1)
            else:
                sku = texto

        # --- Gramos/ml ---
        gramos = "N/A"
        p_gramos = soup.find("p", class_="custom_product__text product__text")
        if p_gramos:
            texto_g = p_gramos.get_text(strip=True)
            match_g = re.search(r"(\d+(?:[.,]\d+)?\s*(?:g|ml|gr|GR|ML))", texto_g, re.IGNORECASE)
            if match_g:
                gramos = match_g.group(1).replace(",", ".").lower()

        driver.quit()
        return sku, gramos

    except Exception as e:
        print(f"⚠️ Error en {url}: {e}")
        try:
            driver.quit()
        except:
            pass
        return "N/A", "N/A"


def ducha():
    """Extrae TODOS los productos de la categoría 'Ducha y Baño' de Nala y devuelve un DataFrame."""

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.get("https://nala.es/collections/ducha-y-bano")

    # Esperar a que se carguen los productos
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.grid__item"))
    )

    # --- Scroll dinámico ---
    last_count = 0
    stable_rounds = 0
    while stable_rounds < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "li.grid__item")
        if len(products) == last_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
        last_count = len(products)

    print(f"Total productos detectados en la página: {last_count}")

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    items = soup.select("li.grid__item")
    print(f"Productos encontrados en HTML: {len(items)}")

    productos = []
    categorias = defaultdict(int)

    for item in items:
        enlace = item.find("a", href=re.compile("/products/"))
        if not enlace:
            continue
        url_producto = "https://nala.es" + enlace.get("href")

        # ====== 🔍 Extracción del nombre ======
        nombre = None
        nombre_tag = item.find(attrs={"class": re.compile("(title|heading|card__title)", re.IGNORECASE)})
        if nombre_tag and nombre_tag.get_text(strip=True):
            nombre = nombre_tag.get_text(strip=True)

        if not nombre:
            hidden_name = item.find("span", class_=re.compile("visually-hidden", re.IGNORECASE))
            if hidden_name:
                nombre = hidden_name.get_text(strip=True)

        if not nombre:
            nombre = enlace.get_text(strip=True)

        if not nombre:
            nombre = "N/A"
        # =====================================================

        # Precio
        precio_tag = item.find("div", class_=re.compile("price", re.IGNORECASE))
        precio = "N/A"
        if precio_tag:
            precio_texto = precio_tag.get_text(strip=True)
            precio_match = re.search(r"\d+,\d{2}", precio_texto)
            if precio_match:
                precio = precio_match.group() + " €"

        # Ingrediente clave
        ingrediente = re.search(
            r"(granada|café|vainilla|aloe|argán|karité|jojoba|caléndula|manzanilla|aguacate|cáñamo|rosas|lavanda|menta|coco|limón|melocotón|piña|mango|pepino|pomelo|uvas|naranja)",
            nombre,
            re.IGNORECASE
        )

        # Clasificación por categoría
        nombre_lower = nombre.lower()
        if "aceite de ducha" in nombre_lower:
            categoria = "Aceite de Ducha"
        elif "manos" in nombre_lower:
            categoria = "Jabon de manos"
        elif "ducha" in nombre_lower or "gel" in nombre_lower:
            categoria = "Gel de Ducha"
        elif "bomba" in nombre_lower:
            categoria = "Bomba de Baño"
        elif "crema" in nombre_lower:
            categoria = "Crema de Baño"
        elif "espuma" in nombre_lower:
            categoria = "Espuma de Baño"
        elif "sal" in nombre_lower:
            categoria = "Sal de Baño"
        elif "leche" in nombre_lower:
            categoria = "Leche de Baño"
        elif "natural" in nombre_lower:
            categoria = "Jabon Natural"
        else:
            categoria = "Otro"

        categorias[categoria] += 1

        productos.append({
            "SKU": "N/A",        # se completará luego
            "gramos/ml": "N/A",  # nuevo campo
            "nombre": nombre,
            "categoria_general": "Ducha y Baño",
            "categoria": categoria,
            "precio": precio,
            "ingrediente_clave": ingrediente.group(1).capitalize() if ingrediente else "N/A",
            "país": "España",    # nuevo campo
            "url": url_producto,
        })

    # === Extraer SKUs y gramos/ml en paralelo ===
    print("Extrayendo SKUs y gramos/ml de cada producto...")
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extraer_sku_y_gramos, p["url"]): i for i, p in enumerate(productos)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                sku, gramos = future.result()
                productos[idx]["SKU"] = sku
                productos[idx]["gramos/ml"] = gramos
            except Exception:
                productos[idx]["SKU"] = "N/A"
                productos[idx]["gramos/ml"] = "N/A"

    df = pd.DataFrame(productos)
    print(f"Total productos finales: {len(df)}")
    # === Convertir precio y gramos/ml a columnas numéricas ===
    df["precio"] = (
        df["precio"]
        .str.replace("€", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
        )

    df["gramos/ml"] = (
        df["gramos/ml"]
        .str.extract(r"(\d+(?:\.\d+)?)")[0]   # Extrae el número
        .astype(float)                        # Convierte a decimal
        .round()                              # Redondea si hay decimales
        .astype("Int64")                      # Convierte a entero permitiendo NaN
    )

    return df


In [41]:
df_ducha = ducha()
df_ducha

Total productos detectados en la página: 126
Productos encontrados en HTML: 126
Extrayendo SKUs y gramos/ml de cada producto...
Total productos finales: 122


,SKU,gramos/ml,nombre,categoria_general,categoria,precio,ingrediente_clave,país,url
0,35735,200,Gel de Ducha Exfoliante Hidratante - Plátano &...,Ducha y Baño,Gel de Ducha,7.9,N/A,España,https://nala.es/products/gel-ducha-exfoliante-...
1,35490,125,Bomba de Baño Efervescente - Pera & Sandía,Ducha y Baño,Bomba de Baño,3.9,N/A,España,https://nala.es/products/bomba-bano-efervescen...
2,35155,200,Gel de Ducha - Melocotón & Mango,Ducha y Baño,Gel de Ducha,5.9,Melocotón,España,https://nala.es/products/gel-de-ducha-melocoto...
3,35162,200,Gel de Ducha - Piña & Sándalo,Ducha y Baño,Gel de Ducha,5.9,Piña,España,https://nala.es/products/gel-de-ducha-pina-san...
4,35148,200,Gel de Ducha - Fruta de la Pasión & Litchi,Ducha y Baño,Gel de Ducha,5.9,N/A,España,https://nala.es/products/gel-de-ducha-fruta-de...
...,...,...,...,...,...,...,...,...,...
117,64446,125,Bomba de Baño - Fresa Dulce,Ducha y Baño,Bomba de Baño,3.9,N/A,España,https://nala.es/products/bomba-de-bano-dulce-f...
118,65034,200,Gel de ducha - Melocotón,Ducha y Baño,Gel de Ducha,5.9,Melocotón,España,https://nala.es/products/gel-de-ducha-melocoton
119,64445,125,Bomba de baño - Cítricos,Ducha y Baño,Bomba de Baño,3.9,N/A,España,https://nala.es/products/bomba-de-bano-citrus
120,70418,100,Jabón natural - Ylang Ylang,Ducha y Baño,Jabon Natural,2.9,N/A,España,https://nala.es/products/jabon-natural-ylang-y...


In [42]:
df_combinado = pd.concat([df_cabello, df_coproral, df_ducha, df_rostro], ignore_index=True)

df_combinado.to_csv("nala_es.csv", index=False)